# Incompressible flow with obstacle and large Reynolds number

We consider the unsteady Navier-Stokes problem defined by the following set of differential equations: 

$$
\begin{cases}
  \frac{\partial \boldsymbol{u}}{\partial t} +
  ({\boldsymbol{u}}\cdot\boldsymbol{\nabla})\ \boldsymbol{u}
  -\dfrac{1}{\text{Re}}\boldsymbol{\Delta} \boldsymbol{u} + \boldsymbol{\nabla} p = 0, 
  \qquad&\text{in }\Omega\times I, \\
  \boldsymbol{\nabla} \cdot \boldsymbol{u} = 0, 
  \qquad &\text{in }\Omega\times I, \\
  \boldsymbol{u} = (1-y^2) \boldsymbol{i}, \qquad&\text{on }(\Gamma_{\text{in}} := \{x= -1,\, -1\le y \le 1 \})\times I, \\
  \left(\dfrac{1}{\text{Re}} \boldsymbol{\nabla} \boldsymbol{u} - pI\right) \boldsymbol{n} = 0, \qquad&\text{on }(\Gamma_{\text{out}} := \{x=4,\, -1\le y \le 1\})\times I, \\
  \boldsymbol{u} = \boldsymbol{0}, \qquad&\text{on }\partial\Omega \backslash (\Gamma_{\text{in}}\cup\Gamma_{\text{out}}) \times I, \\
  \boldsymbol{u}(\cdot,t=0) = \boldsymbol{0}, &\text{in }\Omega,
\end{cases}
$$

where the Reynolds number $\text{Re}$ is equal to $1000$, the time interval is $I=(0,40)$, and the spacial domain $\Omega$ is described in the following figure.

![alt text](https://drive.google.com/uc?export=view&id=190IgTTTbGtbyWKwFfemQlQWOAOAdbNh-)

## FEniCS solution

In [1]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
    
from fenics import *
from mshr import *

### Fractional step Chorin-Temam method

The Chorin-Temam formulation consists in the following steps:
given the solution $(\boldsymbol{u}_n,p_n)$ at step $n$,

*   Solve the predictor step
    $$
  \begin{cases}
  {\delta t}^{-1}\boldsymbol{\tilde{u}} +
  ({\boldsymbol{u}_n}\cdot\boldsymbol{\nabla})\ 
  \boldsymbol{\tilde{u}} -
  \text{Re}^{-1}\boldsymbol{\Delta}\boldsymbol{\tilde{u}} 
  = -\boldsymbol{\nabla} p_n + {\delta t}^{-1}\boldsymbol{u}_n, 
  \qquad &\text{in }\Omega, \\
  \boldsymbol{\tilde{u}} = (1-y^2) \boldsymbol{i}, 
  \qquad&\text{on }\Gamma_{\text{in}}, \\
  (\boldsymbol{\nabla}\boldsymbol{\tilde{u}}) \boldsymbol{n}=0,
  \qquad&\text{on }\Gamma_{\text{out}}, \\
  \boldsymbol{\tilde{u}} = \boldsymbol{0}, 
  \qquad&\text{on }\partial\Omega\backslash 
  (\Gamma_{\text{in}}\cup\Gamma_{\text{out}}).
  \end{cases}
    $$

*   Solve the Poisson problem for the pressure
    $$
  \begin{cases}
  -\Delta \delta p = - 
  {\delta t}^{-1}\boldsymbol{\nabla}\cdot\boldsymbol{\tilde{u}},
  \qquad&\text{in }\Omega, \\
  \delta p = 0, \qquad&\text{on }\Gamma_{\text{out}}, \\
  \boldsymbol{\nabla}{\delta p}\cdot\boldsymbol{n} = 0,
  \qquad&\text{on }\partial\Omega\backslash\Gamma_{\text{out}}.
  \end{cases}
    $$ 

*   Velocity and pressure updates
  $$
  \boldsymbol{u}_{n+1} = \tilde{\boldsymbol{u}} 
  - \delta t \boldsymbol{\nabla}{\delta p}, \qquad\quad
  p_{n+1} = p_{n} + \delta p.
    $$


Applying this method often results in inaccurate approximations because of the **artificial boundary conditions** introduced to solve the predictor and pressure steps. In particular, we can observe that **we are not able to control the tangential component of the velocity** on the boundary. This often leads to unphysical solutions. 

Nevertheless, the idea of splitting the solution of the linear system in different steps has inspired the formulation of **algebraic fractional step schemes**.

### Algebraic fractional step method

We discretize the problem using the **stable FE pair $\mathbb{P}^2-\mathbb{P}^1$** and solve it by using the **algebraic Chorin-Temam projection scheme coupled with the backward Euler semi-implicit scheme**.

Using the incremental form ($\delta\boldsymbol{P}_{n+1} = \boldsymbol{P}_{n+1}-\boldsymbol{P}_{n}$) at
each step the following linear system has to be solved:

$$
  \begin{bmatrix}C & B^{T} \\ B & 0\end{bmatrix}
  \begin{bmatrix}\boldsymbol{U}_{n+1} \\ 
  \delta\boldsymbol{P}_{n+1}\end{bmatrix}
  =
  \begin{bmatrix}\boldsymbol{f}_{n+1} + \frac{1}{\Delta t} M \boldsymbol{U}_n - B^T \boldsymbol{P}_n \\ \boldsymbol{0} \end{bmatrix},
  \qquad
  C = \frac{1}{\Delta t} M + A + N(\boldsymbol{U}_n).
$$

Rather than solving this system, let us now consider the following **inexact factorization** of the system matrix

$$
  \begin{bmatrix}C & B^{T} \\ B & 0\end{bmatrix}
  =
  \begin{bmatrix}C & 0 \\ B & -B C^{-1} B^T\end{bmatrix}
  \begin{bmatrix}I & C^{-1}B^{T} \\ 0 & I\end{bmatrix}
  \approx
  \begin{bmatrix}C & 0 \\ B & -\Delta t B M^{-1} B^{T}\end{bmatrix}
  \begin{bmatrix}I & \Delta t M^{-1} B^{T} \\ 0 & I\end{bmatrix}.
$$

### Implementation of algebraic Chorin-Temam scheme in FEniCS

First, we need to import the block package, in order to perform block operations.

In [2]:
%%capture
!rm -rf *
!pip install git+https://mbotti@bitbucket.org/mbotti/cbc_mod.block.git
from block import *
from block.dolfin_util import *
from block.iterative import *
from block.algebraic.petsc import *

The numerical scheme is described by the following steps:

*   Solve the predictor step
    $$
    C \tilde{\boldsymbol{U}}_{n+1} = \boldsymbol{f}_{n+1} +
    \frac{1}{\Delta t} M \boldsymbol{U}_n-B^T \boldsymbol{P}_n.
    $$

*   Solve the Poisson problem for the pressure
    $$
    B M^{-1} B^{T} \delta\boldsymbol{P}_{n+1} = 
    \frac{1}{\Delta t} B \tilde{\boldsymbol{U}}_{n+1}.
    $$

*   Update the velocity (projection step)
    $$
    \boldsymbol{U}_{n+1} =  \tilde{\boldsymbol{U}}_{n+1} 
    - \Delta t M^{-1} B^{T} \delta\boldsymbol{P}_{n+1}.
    $$

*   Update the pressure
    $$
      \boldsymbol{P}_{n+1} = \boldsymbol{P}_n 
      + \delta\boldsymbol{P}_{n+1}.
    $$

In [4]:
# 1. mesh generation
n = 120

hole = Circle(Point(0.0, 0.0), 0.2)
box = Rectangle(Point(-1.0, -1.0), Point(4.0, 1.0))
geometry = box - hole
mesh = generate_mesh(geometry, n)

# 2. finite element space
degree = 1
V = VectorFunctionSpace(mesh, 'CG', degree+1)
Q = FunctionSpace(mesh, 'CG', degree)

def inflow(x, on_boundary):
  return on_boundary and near(x[0], -1.0)

def rigid_wall(x, on_boundary):
  return on_boundary and not near(x[0], -1.0) and not near(x[0], 4.0)

u_inflow = Expression(('1 - x[1] * x[1]', '0'), degree=2)

bcs = [DirichletBC(V, u_inflow, inflow),
       DirichletBC(V, Constant((0.0, 0.0)), rigid_wall)]

# 3. problem definition
u = TrialFunction(V)
v = TestFunction(V)

p = TrialFunction(Q)
q = TestFunction(Q)

f = Constant((0.0, 0.0))
Re = Constant(1000)
dt = 0.5

u_old, p_old = Function(V), Function(Q)

c = (dot(u, v) / Constant(dt) + dot(grad(u) * u_old, v) + inner(grad(u), grad(v)) / Re) * dx
bT = (- p * div(v)) * dx
b = (- div(u) * q) * dx
m = dot(u, v) / Constant(dt) * dx

L1 = (dot(u_old, v) / Constant(dt) + dot(f, v)) * dx

# 4. solution
time = 0.0

u, p = Function(V), Function(Q)
u.rename('velocity', 'velocity')
p.rename('pressure', 'pressure')

ufile = File('velocity.pvd')
pfile = File('pressure.pvd')

ufile << (u, time)
pfile << (p, time)

BT = assemble(bT)
for bc in bcs:
  bc.zero(BT)
B = assemble(b)

# Dirichlet BCs for the velocity field must be applied also to $M$!
# We replace M with its lumped version $M_L$ to speed up the solver
M = assemble(m)
for bc in bcs:
  bc.apply(M)
Minv = LumpedInvDiag(M)

while time < 40.0:
  time += dt
  f.t = time

  u_old.assign(u)
  p_old.assign(p)
  
  # first step: velocity prediction
  rhs = assemble(L1) - BT * p_old.vector()

  C = assemble(c)
  for bc in bcs:
    bc.apply(C, rhs)

  Cinv = LU(C)
  u_tilde = Cinv * rhs
  
  # second step: compute pressure
  rhs = B * u_tilde
  
  S = B * Minv * BT
  Sinv = LGMRES(S)
  delta_p = Sinv * rhs
  
  # third step: update velocity and pressure
  u_tilde -= Minv * BT * delta_p
  
  u.vector().set_local(u_tilde.get_local())
  p.vector().set_local(p_old.vector().get_local() + delta_p.get_local())
  
  ufile << (u, time)
  pfile << (p, time)

LGMRES converged [iter=20, time=4.05s, res=9.8e-06]
LGMRES converged [iter=20, time=4.03s, res=7.6e-06]
LGMRES converged [iter=21, time=5.13s, res=7.5e-06]
LGMRES converged [iter=20, time=4.04s, res=9.0e-06]
LGMRES converged [iter=20, time=4.02s, res=6.7e-06]
LGMRES converged [iter=15, time=3.13s, res=8.5e-06]
LGMRES converged [iter=21, time=4.22s, res=5.3e-06]
LGMRES converged [iter=21, time=4.25s, res=3.8e-06]
LGMRES converged [iter=19, time=3.88s, res=9.6e-06]
LGMRES converged [iter=16, time=3.74s, res=8.8e-06]
LGMRES converged [iter=18, time=3.65s, res=9.6e-06]
LGMRES converged [iter=13, time=2.66s, res=8.2e-06]
LGMRES converged [iter=13, time=2.64s, res=5.7e-06]
LGMRES converged [iter=13, time=2.63s, res=8.5e-06]
LGMRES converged [iter=12, time=2.42s, res=6.8e-06]
LGMRES converged [iter=13, time=2.64s, res=7.9e-06]
LGMRES converged [iter=12, time=2.48s, res=1.0e-05]
LGMRES converged [iter=11, time=2.25s, res=7.4e-06]
LGMRES converged [iter=12, time=2.43s, res=5.1e-06]
LGMRES conve